In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from ipywidgets import interactive

import numpy as np
import scipy

import soundfile as sf
from utils import *

# Índice

1. [Serviços de Sinal Horário](#sinalHorario)
    1.1 [MSF](#msf1)
    1.2 [DCF77](#DCF771)
2. [Codificação do sinal horário](#cod)
    2.1 [MSF](#msf2)
    2.2 [DCF77](#DCF772)
3. [Explorações](#exp)
4. [Links](#link)

## 1. Serviços de Sinal Horário

<a id="sinalHorario"></a>

Na Europa, existem dois transmissores de sinal horario que operam com frequâncias suficientemente baixas para conseguirmos registar o seu sinal usando a placa de som do computador. São eles o [MSF](https://en.wikipedia.org/wiki/Time_from_NPL_(MSF)), localizado no Reino Unido e que opera a $60kHz$, e o [DCF77](https://en.wikipedia.org/wiki/DCF77), localizado na Alemanha e a operar a $77.5kHz$.

In [ ]:
make_spect_plot('media/obser_001.wav', nptf=4096, overlap=0.5, fmin=55000, fmax=80000)

In [ ]:
Viseu = (40.6582338,-7.9123233)

### 1.1 MSF
<a id="msf1"></a>

In [ ]:
MSF = (54.911195,-3.279302)

In [ ]:
d1 = distancia(MSF, Viseu)

In [ ]:
a1 = azimute( MSF, Viseu)

In [ ]:
mapa_2pontos_eur(MSF, Viseu, nomes=['MSF','Viseu'])

### 1.2 DCF77

<a id="DSF771"></a>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Dcf77.jpg/640px-Dcf77.jpg" alt="Drawing"  style="width: 50%;"/>

[fonte](https://en.wikipedia.org/wiki/File:Dcf77.jpg)

In [ ]:
DCF77 = (50.015411,9.008307)

In [ ]:
d2 = distancia(DCF77, Viseu)

In [ ]:
a2 = azimute(DCF77, Viseu)

In [ ]:
mapa_2pontos_eur(DCF77, Viseu, nomes=['DCF77','Viseu'])

## 2. Codificação do sinal horário

<a id="cod"></a>

### 2.1 MSF
<a id="msf2"></a>

Para mais informação ver os segintes links: [links](https://en.wikipedia.org/wiki/Time_from_NPL_(MSF)), [links](http://www.npl.co.uk/science-technology/time-frequency/)

In [ ]:
make_spect_plot_H('media/obser_001.wav', nptf=4096, overlap=0.90, fmin=59500, fmax=60500, tmin=0.15, tmax=1.2)

### 2.2 DSF77

<a id="DSF772"></a>

Para mais informação ver os segintes links: [links](https://en.wikipedia.org/wiki/DCF77),   [links](http://www.ptb.de/cms/en/ptb/fachabteilungen/abt4/fb-44/ag-442/dissemination-of-legal-time/dcf77.html)

In [ ]:
make_spect_plot_H('media/obser_001.wav', nptf=4096, overlap=0.90, fmin=77000, fmax=78000, tmin=0.15, tmax=1.2)

## 3. Explorações

<a id="exp"></a>

In [ ]:
make_spect_plot('media/MSF26032010.wav', nptf=4096, overlap=0.5, fmin=55000, fmax=80000)

In [ ]:
make_spect_plot_H('media/MSF26032010.wav', nptf=4096, overlap=0.90, fmin=59500, fmax=60500, tmin=0.15, tmax=1.2)

# 4. Links 

<a id="links"></a>

1. [ Sudden Ionospheric Disturbances Monitoring Station A118](https://sidstation.loudet.org/)

2. [vlf.it](http://www.vlf.it/)

3. [Construção de recetores rádio como introdução à Física das Telecomunicações - parte I ](https://www.spf.pt/magazines/GFIS/113/913)

4. [Construção de recetores rádio como introdução à Física das Telecomunicações - parte II ](https://www.spf.pt/magazines/GFIS/114/931)

5. [Ouvindo o emissor SAQ de Grimeton ](https://www.spf.pt/magazines/GFIS/118/969)